# Модуль 4.4 — ParentDocumentRetriever

**Цель:** сохранять контекст целого документа, но искать по маленьким чанкам.

**Что сделаем:**
- установим библиотеки
- подготовим документы
- построим ParentDocumentRetriever
- разберем практические советы

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  pydantic==2.12.3 \
  requests==2.32.4

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Подготовка папки `data`

Если папки нет, создадим её и положим пример файла, чтобы всё запускалось сразу.

In [ ]:
from pathlib import Path

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)

sample_path = data_dir / "sample.txt"
if not sample_path.exists():
    sample_path.write_text(
        "LangChain помогает строить цепочки (chains), агенты и RAG‑системы.\n"
        "ParentDocumentRetriever сохраняет контекст родительского документа.\n"
        "Эмбеддинги позволяют искать похожие фрагменты текста.",
        encoding="utf-8",
    )

print("Файлы в data:", [p.name for p in data_dir.glob("*.txt")])

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## ParentDocumentRetriever: идея

Мы индексируем **маленькие чанки**, но возвращаем **родительские документы**, чтобы сохранить больше контекста.
Это особенно полезно, когда ответ должен опираться на большой кусок исходного текста.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

# Загружаем документы
loader = DirectoryLoader(
    "./data",
    glob="**/*.txt",
    loader_cls=TextLoader,
    show_progress=True,
)

documents = loader.load()

# Делаем маленькие чанки для поиска
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

# А «родитель» может быть крупнее
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100)

# Векторное хранилище для маленьких чанков
store = LocalFileStore("./cache/embeddings")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=os.environ.get("OPENAI_BASE_URL"))
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace="openai")

vectorstore = Chroma(
    collection_name="parent_retriever",
    embedding_function=cached_embedder,
    persist_directory="./db/chroma_parent",
)

# Хранилище для родительских документов
parent_store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=parent_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# Индексация
retriever.add_documents(documents)

# Поиск
query = "Что такое RAG?"
results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results, 1):
    print(f"\nДокумент {i}:")
    print(doc.page_content[:400])

## Практические рекомендации

- Делайте `child_chunk` маленьким (200–400) для точного поиска.
- Делайте `parent_chunk` крупнее (800–1500), чтобы вернуть контекст.
- Если ответ обрывается — увеличивайте `parent_chunk`.
- Храните `persist_directory`, чтобы не пересоздавать индекс.